In [1]:
%use kandy


In [2]:
import org.cryptobiotic.rla.*

//// Read overall audit information (including the seed) and contest information

val audit: AuditSimple = AuditSimple(
    cvr_file=      "/home/stormy/dev/github/rla/shangrla-kotlin/src/test/data/rla/SFDA2019_PrelimReport12VBMJustDASheets.raire",
    manifest_file= "/home/stormy/dev/github/rla/shangrla-kotlin/src/test/data/rla/N19.ballotmanifest.VBM.11-14.xlsx",
    max_cards=293555)


In [3]:
//// Read ballot manifest, get total ballots
val manifest = DataFrame.read(audit.manifest_file)
manifest.schema()

val totalBallots = manifest["Total Ballots"] as ValueColumn<Double>
totalBallots.toList().sum()

293555.0

In [4]:
import org.cryptobiotic.shangrla.reader.readRaireBallots
import org.cryptobiotic.shangrla.reader.showRaireBallots

//// Read cvrs.
val raireBallots = readRaireBallots(audit.cvr_file)
showRaireBallots(raireBallots, 11)

RaireContests [RaireContest(name=339, candidates=[15, 16, 17, 18], winner=Contest, tot_ballots=146662, outcome=[])]
Cvrs Records
 Ballot '99813_1_1'= Contest '339': '17':0, 
 Ballot '99813_1_3'= Contest '339': '16':0, 
 Ballot '99813_1_6'= Contest '339': '15':2, '16':3, '17':1, '18':0, 
 Ballot '99813_1_8'= Contest '339': '18':0, 
 Ballot '99813_1_9'= Contest '339': 
 Ballot '99813_1_11'= Contest '339': '15':2, '16':0, '17':1, '18':3, 
 Ballot '99813_1_13'= Contest '339': '15':0, '16':1, '17':2, '18':3, 
 Ballot '99813_1_16'= Contest '339': '15':0, 
 Ballot '99813_1_17'= Contest '339': '15':0, 
 Ballot '99813_1_19'= Contest '339': '16':0, 
 Ballot '99813_1_26'= Contest '339': '16':0, 
 Ballot '99813_1_27'= Contest '339': '15':0, 
 ...


In [5]:

import org.cryptobiotic.shangrla.reader.makeCvrsFromRaireBallots

val cvrs : List<CvrSimple> = makeCvrsFromRaireBallots(raireBallots.second, 11)
cvrs.forEach { println(it) }


CvrSimple(id=99813_1_1, votes={339={17=0}}, phantom=false, sampled=false)
CvrSimple(id=99813_1_3, votes={339={16=0}}, phantom=false, sampled=false)
CvrSimple(id=99813_1_6, votes={339={15=2, 16=3, 17=1, 18=0}}, phantom=false, sampled=false)
CvrSimple(id=99813_1_8, votes={339={18=0}}, phantom=false, sampled=false)
CvrSimple(id=99813_1_9, votes={339={}}, phantom=false, sampled=false)
CvrSimple(id=99813_1_11, votes={339={15=2, 16=0, 17=1, 18=3}}, phantom=false, sampled=false)
CvrSimple(id=99813_1_13, votes={339={15=0, 16=1, 17=2, 18=3}}, phantom=false, sampled=false)
CvrSimple(id=99813_1_16, votes={339={15=0}}, phantom=false, sampled=false)
CvrSimple(id=99813_1_17, votes={339={15=0}}, phantom=false, sampled=false)
CvrSimple(id=99813_1_19, votes={339={16=0}}, phantom=false, sampled=false)
CvrSimple(id=99813_1_26, votes={339={16=0}}, phantom=false, sampled=false)
CvrSimple(id=99813_1_27, votes={339={15=0}}, phantom=false, sampled=false)


In [6]:
//// Skip phantoms, ~2EZ for now. Assume that each CVR has a corresponding manifest entry.
// N_phantoms = max_cards - cards_in_manifest
// so we just set max_cards to cards_in_manifest 
// TODO: 293555.0 vs 146662; python has 293555

val votes: Map<String, Map<String, Int>> = CvrSimple.tabulate_votes(cvrs)
val styles: Map<Set<String>, Int> = CvrSimple.tabulate_styles(cvrs)
val cards: Map<String, Int> = CvrSimple.tabulate_cards_contests(cvrs)

// TODO just use RaireContests?
val contests: List<ContestSimple> = ContestSimple.fromVotes(audit, votes, cards)
contests.forEach { println(it) }


ContestSimple(id=339, name=339, candidates=[17, 16, 15, 18], choice_function=PLURALITY, assertion_file=null, audit_type=CARD_COMPARISON, betFn=null, estimFn=null, testFn=null, g=0.1, n_winners=1, reported_winners=[], risk_limit=0.05, share_to_win=0.5, use_style=true, assertions={}, ncvrs=0, ncards=12, sample_size=null, sample_threshold=null, tally={})


In [10]:
//// + Create Assertions for every Contest, including an Assorter and NonnegMean for every Assertion

    make_all_assertions(contests)


contests.map {
    it.assertions.forEach { println(it) }
}


[kotlin.Unit]

In [8]:
//// Calculate assorter margins for all assorters:
// If `not use_style`, apply the Assorter to all cards and CVRs, including phantoms
//        - Else apply the assorter only to cards/cvrs reported to contain the contest, including phantoms that contain the contest


In [9]:
//// Set `assertion.test.u` to the appropriate value for each assertion: 
// `assorter.upper_bound` for polling audits or `2/(2-assorter.margin/assorter.upper_bound)` for ballot-level comparison audits